<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>

# Week 1: Find Clusters of Infected People

<span style="color:red">
**URGENT WARNING**

We have been receiving reports from health facilities that a new, fast-spreading virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Your goal for this notebook will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [1]:
%load_ext cudf.pandas
import pandas as pd
import cuml

import cupy as cp

## Load Data

Begin by loading the data you've received about week 1 of the outbreak into a cuDF-accelerated pandas DataFrame. The data is located at `'./data/week1.csv'`. For this notebook you will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after loading, or use the `pd.read_csv` named argument `usecols` to provide a list of only the columns you need.

In [2]:
df = pd.read_csv('./data/week1.csv', usecols = ["lat", "long", "infected"])
df = df.astype({"lat":"float32", "long":"float32"})
df.head()
df.dtypes

lat         float32
long        float32
infected       bool
dtype: object

## Make Data Frame of the Infected

Make a new DataFrame `infected_df` that contains only the infected members of the population.

**Tip**: Reset the index of `infected_df` with `.reset_index(drop=True)`. 

In [4]:
infected_df = df.loc[df["infected"] == True].reset_index()
infected_df = infected_df.reset_index(drop=True)

## Make Grid Coordinates for Infected Locations

Provided for you in the next cell (which you can expand by clicking on the "..." and contract again after executing by clicking on the blue left border of the cell) is the lat/long to OSGB36 grid coordinates converter you used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `infected_df` you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [6]:
cupy_lat = cp.asarray(infected_df['lat'])
cupy_long = cp.asarray(infected_df["long"])

infected_df['northing'], infected_df['easting'] = latlong2osgbgrid_cupy(cupy_lat, cupy_long)


# infected_df.dtypes
# lat = cp.asarray(infected_df['lat'])
# long = cp.asarray(infected_df["long"])
# northing_series, easting_series = latlong2osgbgrid_cupy(lat, long)
# infected_df["northing"], infected_df["easting"] = northing_series, easting_series

## Find Clusters of Infected People

Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member. Create a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [10]:
infected_df = infected_df[infected_df['infected'] == 1]
coordinates = infected_df[['northing', 'easting']].values

dbscan = cuml.DBSCAN(eps = 2000, min_samples=25, metric='euclidean')
infected_df['cluster'] = dbscan.fit(infected_df[['northing','easting']])
infected_df['cluster'] = dbscan.fit_predict(coordinates)

infected_df
# infected_df["label"] = dbscan.fit(infected_df[['northing','easting']]).labels_

# Filter out only infected people (if the 'infected' column indicates infection status)
# infected_df = infected_df[infected_df['infected'] == 1]

# Step 1: Get the coordinates (northing and easting) of infected people
# coordinates = infected_df[['northing', 'easting']].values

# Step 2: Apply DBSCAN clustering with the appropriate parameters
# - eps = 2000 (maximum distance between two samples to be considered as in the same cluster)
# - min_samples = 25 (at least 25 people in each cluster)
# db = DBSCAN(eps=2000, min_samples=25, metric='euclidean')

# Step 3: Fit the DBSCAN model to the coordinates of infected people
# infected_df['cluster'] = db.fit_predict(coordinates)

# # The `cluster` column will contain cluster labels, where -1 indicates noise/outliers
# # Note: If you want to exclude outliers (-1), you can filter them out like so:
# # infected_df = infected_df[infected_df['cluster'] != -1]

# # Step 4: Inspect the result
# print(infected_df[['northing', 'easting', 'cluster']])

index        lat      long  infected       northing        easting  \
0      28928759  54.472767 -1.654932      True  508670.609844  422359.755045   
1      28930512  54.529716 -1.667143      True  515003.077959  421538.536701   
2      28930904  54.512985 -1.589866      True  513167.686551  426549.869615   
3      28932226  54.522324 -1.380694      True  514305.903560  440081.214659   
4      28933748  54.541660 -1.613490      True  516349.568138  425002.994784   
...         ...        ...       ...       ...            ...            ...   
18143  57404109  52.428349 -3.322932      True  282016.882768  310060.089226   
18144  57406802  52.415894 -3.263942      True  280559.578075  314046.122410   
18145  57410428  52.539932 -3.617128      True  294832.747453  290338.182711   
18146  57411005  52.435490 -3.597263      True  283187.785292  291428.269805   
18147  57411919  52.700104 -3.375221      True  312306.375259  307081.471424   

       cluster  
0           -1  
1           -1  
2           -1  
3           -1  
4           -1  
...        ...  
18143       -1  
18144       -1  
18145       -1  
18146       -1  
18147       -1  

[18148 rows x 7 columns]

## Find the Centroid of Each Cluster

Use grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [11]:
centroids_df = infected_df[['northing', 'easting', 'cluster']].groupby(['cluster']).mean()
centroids_df

northing        easting
cluster                              
-1       378085.754791  401877.065780
 0       397661.304992  371410.018108
 1       436475.655601  332980.450191
 2       347062.488769  389386.816919
 3       359668.898834  379638.014893
 4       391630.334260  431158.137402
 5       386471.539887  426559.087494
 6       434970.643042  406985.278792
 7       412772.945607  410069.660763
 8       415807.575317  414765.630595
 9       417322.762302  409583.734443
 10      334208.465306  435937.776378
 11      300568.141909  391901.509433
 12      291539.640465  401640.663163
 13      289855.065507  394518.290054

Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [12]:
infected_df['cluster'].value_counts()

cluster
 0     8638
-1     8449
 2      403
 8       94
 12      72
 13      71
 1       68
 11      68
 4       66
 10      64
 5       43
 7       39
 6       27
 3       25
 9       21
Name: count, dtype: int64

northing    436475.655601
easting     435937.776378
dtype: float64

## Find the Centroid of the Cluster with the Most Members ##

Use the cluster label for with the most people to filter `centroid_df` and write the answer to `my_assessment/question_1.json`. 

In [19]:
centroids_df.loc[infected_df['cluster'].value_counts().idxmax()].to_json('my_assessment/question_1.json')

# infected_df['cluster'].value_counts().idxmax()


/opt/conda/lib/python3.10/site-packages/cudf/io/json.py:194: UserWarning: Using CPU via Pandas to write JSON dataset
  warnings.warn("Using CPU via Pandas to write JSON dataset")


## Check Submission ##

In [14]:
!cat my_assessment/question_1.json

{"northing":436475.6556008835,"easting":435937.7763784334}

**Tip**: Your submission file should contain one line of text, similar to: 

```
{'northing':XXX.XX,'easting':XXX.XX}
```

<div align="center"><h2>Please Restart the Kernel</h2></div>

In [58]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<a href="https://www.nvidia.com/dli"><img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/></a>